<a href="https://colab.research.google.com/github/Vadilonga00/Progetto_ML/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Si importano le librerie

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import confusion_matrix, classification_report, recall_score,f1_score
from joblib import dump, load

Si importa il dataset

In [ ]:
data=pd.read_csv('25day_dataset.csv')

Elimino la riga in questione dal momento che l'accensione dei due dispositivi avviene in un solo secondo (possibile errore di campionamento).Non essendo un campione riguardante il test set posso eliminarlo a priori.


In [ ]:
data.drop(1461218, inplace=True) 

Si aggiunge la colonna delle label, in cui per ogni campione viene associata la classe in base ai consumi relativi ai tre elettrodomestici. Sono state introdotte solamente quattro classi dal momento che non vi sono campioni in cui i dispositivi sono accesi contemporaneamente:

0)Tre elettrodomestici spenti

1)Lavatrice accesa

2)Forno acceso

3)Lavastoviglie accesa

In [ ]:
label=[]
for i in range(len(data)):
    if data['wahing_machine'].iloc[i] == 0 and data['oven'].iloc[i] == 0 and data['dishwasher'].iloc[i] == 0:
        label.append(0)
    elif data['wahing_machine'].iloc[i] != 0 and data['oven'].iloc[i] == 0 and data['dishwasher'].iloc[i] == 0:
        label.append(1)
    elif data['wahing_machine'].iloc[i] == 0 and data['oven'].iloc[i] != 0 and data['dishwasher'].iloc[i] == 0:
        label.append(2)
    else:
        data['wahing_machine'].iloc[i] == 0 and data['oven'].iloc[i] == 0 and data['dishwasher'].iloc[i] != 0
        label.append(3)
data['classe']=label

Creazione delle 10 fold di Test

In [ ]:

Test_set1 =data.loc[(data['DateTime']>='2022-01-01') & (data['DateTime']<'2022-01-02')]
Test_set2 =data.loc[(data['DateTime']>='2022-01-02') & (data['DateTime']<'2022-01-03')]
Test_set3 =data.loc[(data['DateTime']>='2022-01-03') & (data['DateTime']<'2022-01-04')]
Test_set4 =data.loc[(data['DateTime']>='2022-01-04') & (data['DateTime']<'2022-01-05')]
Test_set5 =data.loc[(data['DateTime']>='2022-01-05') & (data['DateTime']<'2022-01-06')]
Test_set6 =data.loc[(data['DateTime']>='2022-01-06') & (data['DateTime']<'2022-01-07')]
Test_set7 =data.loc[(data['DateTime']>='2022-01-07') & (data['DateTime']<'2022-01-08')]
Test_set8 =data.loc[(data['DateTime']>='2022-01-08') & (data['DateTime']<'2022-01-09')]
Test_set9 =data.loc[(data['DateTime']>='2022-01-10') & (data['DateTime']<'2022-01-12')]
Test_set10 =data.loc[(data['DateTime']>='2022-01-12') & (data['DateTime']<'2022-01-13 03:17:00')]

Vengono inizializzate due liste vuote in cui verranno inseriti i valori di f_score e recall per ogni classe (per ogni test set).  Ogni training set utilizza tutti i campioni del dataset ad eccezione di quelli utilizzati nel test set corrente. 
Per ogni coppia training-test day verranno restituite per ogni singolo dispositivo le prestazioni sulla f_score e recall.
Per completezza sono state inserite inoltre la matrice di confusione e il classification_report.

In [ ]:
f_score=[]
recall=[]
Test_list=[Test_set1, Test_set2, Test_set3, Test_set4, Test_set5, Test_set6,Test_set7, Test_set8,Test_set9, Test_set10]
for testing_day in range(len(Test_list)) :
    Test_day=pd.DataFrame(Test_list[testing_day])
    Training_day=data.drop(Test_day.index,axis=0)
    
    #Si eliminano i campioni in cui la potenza attiva è minore della potenza consumata dal singolo dispositivo
    inc=Training_day[(Training_day['ActivePower']< Training_day['oven']) | (Training_day['ActivePower']< Training_day['dishwasher']) | (Training_day['ActivePower']< Training_day['wahing_machine'])].index
    Training_day.drop(inc, axis=0)
    
    X_train=Training_day.drop(['DateTime','oven', 'dishwasher', 'wahing_machine','ReactivePower','classe'],axis=1)
    Y_train=Training_day.iloc[:,-1]
    
    #Viene eseguita una compressione del training set prendendo un campione ogni 5 secondi. 
    #Ogni singolo campione possiederà come valori delle features il valore medio calcolato dai 5 campioni presi.
    xtrain=X_train.groupby(np.arange(len(X_train))//5).mean()
    #Analogo ragionamento viene applicato alla colonna delle label, solamente che la metrica statistica utilizzata è la moda
    ytrain=Y_train.groupby(np.arange(len(Y_train))//5).agg(lambda x:x.value_counts().index[0])
    
   
    X_test=Test_day.drop(['DateTime','oven', 'dishwasher', 'wahing_machine','ReactivePower','classe'],axis=1)
    Y_test=Test_day.iloc[:,-1]

    
    #Per evitare lo sbilanciamento delle classi è stata realizzata una tecnica di undersampling sul training set per ridurre i campioni di classe maggioritaria.
    classe0,classe1,classe2,classe3=ytrain.value_counts(sort=False)
    sampling_strategy = {0: 10000, 1: 10000, 2: classe2, 3: classe3}
    rus = RandomUnderSampler(sampling_strategy=sampling_strategy)
    X_res, y_res = rus.fit_resample(xtrain, ytrain)
    
    
    #Si addestra il modello con i parametri ottenuti utilizzando la griglia GridSearchCv e si salvano i modelli per ogni singola fold
    modello=RandomForestClassifier(n_estimators=60, max_depth=20, max_features='sqrt', random_state=1)
    modello.fit(X_res,y_res)
    dump(modello, f'./Modello/model{testing_day+1}.sav')
    pred=modello.predict(X_test)
    
    
    #Si valutano le prestazioni del classificatore addestrato, utilizzando i diversi test set
    r = recall_score(Y_test,pred, average= None) 
    f = f1_score(Y_test,pred, average= None)
    f_score.append(f)
    recall.append(r)
    
    print("CM:\n" + str(confusion_matrix(Y_test,pred)) + "\n")
    print(classification_report(Y_test,pred))

C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: Unde

CM:
[[79419   489    15  1742]
 [  100  3953     0     0]
 [  143     0   539     0]
 [    0     0     0     0]]

              precision    recall  f1-score   support

           0       1.00      0.97      0.98     81665
           1       0.89      0.98      0.93      4053
           2       0.97      0.79      0.87       682
           3       0.00      0.00      0.00         0

    accuracy                           0.97     86400
   macro avg       0.71      0.68      0.70     86400
weighted avg       0.99      0.97      0.98     86400



C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: Unde

CM:
[[76745   917   135   472]
 [  474  6533     0     1]
 [   58     0  1065     0]
 [    0     0     0     0]]

              precision    recall  f1-score   support

           0       0.99      0.98      0.99     78269
           1       0.88      0.93      0.90      7008
           2       0.89      0.95      0.92      1123
           3       0.00      0.00      0.00         0

    accuracy                           0.98     86400
   macro avg       0.69      0.72      0.70     86400
weighted avg       0.98      0.98      0.98     86400

CM:
[[78189   920   821  1165]
 [   47   967    10     0]
 [    0     1   703     0]
 [  375   126     0  3076]]

              precision    recall  f1-score   support

           0       0.99      0.96      0.98     81095
           1       0.48      0.94      0.64      1024
           2       0.46      1.00      0.63       704
           3       0.73      0.86      0.79      3577

    accuracy                           0.96     86400
   macro av

C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: Unde

...

C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CM:
[[166005    946    599    681]
 [   123   2667      0      0]
 [   597      0   1182      0]
 [     0      0      0      0]]

              precision    recall  f1-score   support

           0       1.00      0.99      0.99    168231
           1       0.74      0.96      0.83      2790
           2       0.66      0.66      0.66      1779
           3       0.00      0.00      0.00         0

    accuracy                           0.98    172800
   macro avg       0.60      0.65      0.62    172800
weighted avg       0.99      0.98      0.99    172800



C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CM:
[[89734   498   278  1943]
 [  262  4099     0     0]
 [   36     1  1369     0]
 [    0     0     0     0]]

              precision    recall  f1-score   support

           0       1.00      0.97      0.98     92453
           1       0.89      0.94      0.92      4361
           2       0.83      0.97      0.90      1406
           3       0.00      0.00      0.00         0

    accuracy                           0.97     98220
   macro avg       0.68      0.72      0.70     98220
weighted avg       0.99      0.97      0.98     98220



C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Utente\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: Unde

Una volta trovati i valori per recall e f1_score dei singoli dispositivi (per ogni test set) vado a calcolare i valori medi per ottenere le metriche desiderate.

In [ ]:
#f_score_res=pd.DataFrame(f_score).replace(0,np.nan)
#print("F1_score_Class:\n" + str(f_score_res.mean(axis=0, skipna=True)))
#recall_res=pd.DataFrame(recall).replace(0,np.nan)
#print("\nRecall_Class:\n" + str(recall_res.mean(axis=0, skipna=True)))

Si calcola infine la recall media e la f1_score media

In [ ]:
#f_score_res=pd.DataFrame(f_score).mean(skipna=True)

In [ ]:
#f_score_res=pd.DataFrame(f_score).replace(0,np.nan)
#print("F1_score_Mean:\n" + str((f_score_res.mean(axis=0, skipna=True)).mean()))
#recall_res=pd.DataFrame(recall).replace(0,np.nan)
#print("\nRecall_Mean:\n" + str((recall_res.mean(axis=0, skipna=True)).mean()))